In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from simple_radio_script_encapsulated import _get_teaser_for_story, _get_statements_for_story_id
from simple_radio_script_encapsulated import teaser_matches_crit, aggregate_decisions
from IPython.display import display, Markdown

In [3]:
story_no = 22192 # 23183 # 23188 # 22192 # 23168

In [4]:
t1 = _get_teaser_for_story(story_no)

In [5]:
statements = _get_statements_for_story_id(story_no)

In [6]:
crit_match_prompt = """
    You are a professional journalist in the medicine domain. You will receive a text and i want you to decide whether the following criterion matches the text.
    To help you guide your decision, you will also get a few expert statements in JSON Form.

    Please answer in the following format:
    Criterion
    Decision: yes or no. 
    Reference: Paragraph in the text which matches the criterion.

    For the references find all text passages and quote them in the form of a list.
    Indicate for all text passages whether they come from the text or from an expert statement. 
    If it is a statement, name the expert who made the statement. It is very important to me that the people are explicitly named.
    
    ```
    CRITERION:
    {CRITERIA}
    ```
    
    ```
    TEXT: 
    {TEASER}
    ```
    
    ```
    EXPERT STATEMENTS: 
    {STATEMENTS}
    ```
    Please answer in German.
    """ 

In [7]:
crit1 = """Der Text erwähnt den Evidenzlevel. 
Was sind die Einschränkungen bei den Studienergebnissen? 
Gibt es Einschränkungen, die dazu führen, dass die zentralen Ergebnisse fragwürdig sind? 
"""
crit2 = "The text contains information about cancer"
crit3 = "The text contains mentions a specifoc journal name of a study"
crit4 = "The text describes a mathematical method for optimizing a loss function with four objectives"
crit6 = """Der Text erwähnt das zentrale Forschungsergebnis"""

crits = [crit1, crit2, crit3, crit4, crit6]

In [8]:
outputs = [teaser_matches_crit(t1, str(statements), crit_match_prompt, c) for c in crits]

In [9]:
outputs[0]

'KRITERIUM:\nDer Text erwähnt den Evidenzlevel. Was sind die Einschränkungen bei den Studienergebnissen? Gibt es Einschränkungen, die dazu führen, dass die zentralen Ergebnisse fragwürdig sind?\n\nENTSCHEIDUNG: Ja.\n\nREFERENZ:\n- "Das Verfahren sei zwar recht flexibel, allerdings auch leicht durch kurzfristige Zufallsschwankungen beeinflussbar [I]." (aus dem Text)\n- "Die Autoren schätzen diese Werte daher mithilfe statistischer Verfahren aus den Daten vergleichbarer Länder mit besserer Informationsbasis. Die sich aus dem Verfahren ergebende globale Übersterblichkeit von etwa 14,8 Millionen Todesfällen in den Jahren 2020 und 2021 ist daher mit einer gewissen Unsicherheit verbunden, sollte aber von der richtigen Größenordnung sein." (Aussage von Prof. Dr. Christoph Rothe)\n- "Diese Werte beruhen auf einem statistischen Verfahren, das leider recht stark durch zufällige Schwankungen in den Sterbezahlen vor der Pandemie beeinflusst werden kann." (Aussage von Prof. Dr. Christoph Rothe)\n- 

In [18]:
agg_decisions_prompt = """
    I will provide a list of Texts to you. These Texts are decisions for specified criteria on a news article. 
    I want you to build a table with the three columns CRITERION, MATCH and REFERENCE.
    The Reference should be displayable as an unordered list in Markdown.
    Each Criterion should be in exactly one row.
    The criterea are separated by "|"
    

    Here is the list of the decisions:
    ```
    {TEXT}
    ```

    Do not combine or split any criterion.
    Please answer in German.
"""

    #The column critera should name the specified criteria, match indicates if the criteria matches or not and the column reference shows the part of the text which corresponds to the criteria. 


In [19]:
%time table_out = aggregate_decisions(outputs, agg_decisions_prompt)

CPU times: user 45.5 ms, sys: 1.35 ms, total: 46.8 ms
Wall time: 51.7 s


In [20]:
file_path=str(story_no)+'.md'
with open(file_path, 'w') as file:
    print(table_out, file = file)

In [21]:
display(Markdown(table_out))

| KRITERIUM | ENTSCHEIDUNG | REFERENZ |
|-----------|--------------|----------|
| Der Text erwähnt den Evidenzlevel. Was sind die Einschränkungen bei den Studienergebnissen? Gibt es Einschränkungen, die dazu führen, dass die zentralen Ergebnisse fragwürdig sind? | Ja | <ul><li>"Das Verfahren sei zwar recht flexibel, allerdings auch leicht durch kurzfristige Zufallsschwankungen beeinflussbar [I]." (aus dem Text)</li><li>"Die Autoren schätzen diese Werte daher mithilfe statistischer Verfahren aus den Daten vergleichbarer Länder mit besserer Informationsbasis. Die sich aus dem Verfahren ergebende globale Übersterblichkeit von etwa 14,8 Millionen Todesfällen in den Jahren 2020 und 2021 ist daher mit einer gewissen Unsicherheit verbunden, sollte aber von der richtigen Größenordnung sein." (Aussage von Prof. Dr. Christoph Rothe)</li><li>"Diese Werte beruhen auf einem statistischen Verfahren, das leider recht stark durch zufällige Schwankungen in den Sterbezahlen vor der Pandemie beeinflusst werden kann." (Aussage von Prof. Dr. Christoph Rothe)</li><li>"In weiten Teilen Afrikas und in Teilen Asiens und Südamerikas werden Todesfälle jedoch nur sehr unzuverlässig registriert und Übersterblichkeit kann daher nicht aufgrund von gemeldeten Todesfällen geschätzt werden, sie wird indirekt aufgrund von Korrelationen zwischen Übersterblichkeit und vorhandenen Variablen wie dem Bruttoinlandsprodukt geschätzt." (Aussage von Jonas Schöley)</li><li>"Diese Unsicherheit ist höher bei kleinen Ländern (zum Beispiel Dänemark) und bei Ländern ohne direkte Informationen zu der Anzahl der Verstorbenen." (Aussage von Jonas Schöley)</li><li>"Jenseits dieser statistischen Unsicherheit gibt es aber immer auch einen Rest Modellierungsunsicherheit, so auch beim WHO-Modell." (Aussage von Jonas Schöley)</li><li>"Eine globale Schätzung wie diese ist problematisch, es greifen die verschiedensten Limitationen. Die Autoren gehen selbst auf einige dieser Limitationen ein (Seite sechs, rechte Spalte)." (Aussage von Prof. Dr. Hanno Ulmer)</li><li>"Letztlich handelt es sich jedoch um eine sehr grobe Schätzung, die für einzelne Länder nicht zutreffend sein muss." (Aussage von Prof. Dr. Hanno Ulmer)</li></ul> |
| Der Text enthält Informationen über Krebs. | Nein | Der gesamte Text sowie die Expertenaussagen beziehen sich ausschließlich auf die Übersterblichkeit im Zusammenhang mit der COVID-19-Pandemie und enthalten keine Informationen über Krebs. |
| Der Text enthält die Nennung eines spezifischen Zeitschriftennamens einer Studie. | Ja | <ul><li>"In der Fachzeitschrift „Nature“ ist am 14.12.2022 eine aktualisierte Schätzung der Weltgesundheitsorganisation (WHO) zur globalen Übersterblichkeit bedingt durch COVID-19 (siehe Primärquelle) erschienen." (aus dem Text)</li><li>"Die nun in „Nature“ publizierte Statistik der WHO ist eine Aktualisierung der Ergebnisse, die bereits im Mai dieses Jahres veröffentlicht wurden." (aus dem Text)</li><li>Prof. Dr. Christoph Rothe: "„Die „Nature“-Studie beschreibt das von der WHO verwendete Verfahren zur Berechnung der globalen Übersterblichkeit während der COVID-19-Pandemie in den Jahren 2020 und 2021, und seine Ergebnisse." (aus einer Expertenaussage)</li></ul> |
| Der Text beschreibt eine mathematische Methode zur Optimierung einer Verlustfunktion mit vier Zielen. | Nein | Keine der genannten Referenzen beschreibt eine mathematische Methode zur Optimierung einer Verlustfunktion mit vier Zielen. Stattdessen wird allgemein von statistischen Verfahren und Modellen gesprochen, die zur Schätzung der Übersterblichkeit verwendet werden. |
| Der Text erwähnt das zentrale Forschungsergebnis. | Ja | <ul><li>"Demnach kam es in den Jahren 2020 und 2021 weltweit zu rund 14,8 Millionen zusätzlichen Todesfällen." (aus dem Text)</li><li>"Die durchschnittliche globale Pro-Kopf-Übersterblichkeitsrate im Jahr 2020 lag den Forschenden zufolge bei 0,06 Prozent, 2021 stieg sie auf 0,13 Prozent." (aus dem Text)</li><li>"In Deutschland starben der WHO-Statistik zufolge rund 122.000 Menschen mehr, als zu erwarten gewesen wäre." (aus dem Text)</li><li>Prof. Dr. Christoph Rothe: "Die sich aus dem Verfahren ergebende globale Übersterblichkeit von etwa 14,8 Millionen Todesfällen in den Jahren 2020 und 2021 ist daher mit einer gewissen Unsicherheit verbunden, sollte aber von der richtigen Größenordnung sein." (aus einer Expertenaussage)</li><li>Jonas Schöley: "In dem angehängten Artikel werden für Deutschland für die Jahre 2020/21 rund 122.000 (101.000–143.000) vorzeitig Verstorbene geschätzt." (aus einer Expertenaussage)</li></ul> |

In [14]:
stories = [22192, 23183, 23188, 22192, 23168]
stories = None
# PAY ATTENTION HERE! Costs a lot of time and money.

In [16]:
mediendoktor = {}
if stories:
    for story_no in stories:
        teaser = _get_teaser_for_story(story_no)
        statements = _get_statements_for_story_id(story_no)
        outputs = [teaser_matches_crit(t1, str(statements), crit_match_prompt, c) for c in crits]
        table_out = aggregate_decisions(outputs, agg_decisions_prompt)
        mediendoktor[story_no]=table_out

In [17]:
if stories:
    for story_no in stories:
        print(f"Hier ist die Einschätzung der Kriterien zur Story mit der ID {story_no}:")
        display(Markdown(mediendoktor[story_no]))